# Load libraries & Import data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime

/Users/jasonhuang/anaconda/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_csv("crsp.csv", low_memory = False)

In [3]:
df.head()

,PERMNO,date,SICCD,PRC,VOL,RET,SHROUT
0,10000,19851231,NaN,NaN,NaN,NaN,NaN
1,10000,19860131,3990,-4.3750,1771.0,C,3680.0
2,10000,19860228,3990,-3.2500,828.0,-0.257143,3680.0
3,10000,19860331,3990,-4.4375,1078.0,0.365385,3680.0
4,10000,19860430,3990,-4.0000,957.0,-0.098592,3793.0


In [4]:
#No NAs
df = df.dropna()
df = df[df['RET'] != 'C']

In [6]:
Date = np.unique(df["date"])

In [13]:
Date_recent = Date[Date > 20040101]

# Functions

In [40]:
def average_return(returns):
    n = len(returns)
    returns = returns + np.ones(n)
    avg_return = np.power(np.prod(np.array(returns)),1/n) - 1  
    return avg_return

In [47]:
def Momentum(data, dates, J, K):
    n = len(dates)
    momentum_returns = []
    # Look back J months
    for i in range(J+1,n-K):
        temp_data = data[(data["date"] >= dates[i-J-1]) & (data["date"] <= dates[i+K])]
        permno = np.unique(temp_data["PERMNO"])
        past_returns = []
        past_permno = []
        for j in permno:
            try:
                permno_data = temp_data[(temp_data["PERMNO"]) == j & (temp_data["date"] < dates[i])]
                past_returns.append(average_return(permno_data["RET"]))
                past_permno.append(j)
            except:
                continue
        J_data = pd.DataFrame({'RET':past_returns, 'PERMNO':past_permno})
    
        # Sort Winner and Loser Deciles
        rows = len(J_data['RET'])
        decile = int(np.ceil(rows*0.1))
        J_data = J_data.sort_values(['RET'],ascending = 0)
        J_buy = J_data.head(decile)
        J_sell = J_data.tail(decile)
        
        # Long winners and Short Losers K months forward
        winner_returns = []
        loser_returns = []
        for w in J_buy["PERMNO"]:
            winner_data = temp_data[(temp_data["PERMNO"]) == w & (temp_data["date"] > dates[i])]
            winner_returns.append(average_return(winner_data["RET"]))
        for l in J_sell["PERMNO"]:
            loser_data = temp_data[(temp_data["PERMNO"]) == l & (temp_data["date"] > dates[i])]
            loser_returns.append(average_return(loser_data["RET"]))
        momentum_returns.append(np.sum(winner_returns)/len(J_buy["RET"]) - np.sum(loser_returns)/len(J_sell["RET"]))
    
    # Get final averaged rolling momentum returns
    averaged_momentum_returns = []
    m = len(momentum_returns)
    for k in range(K, m-K):
        temp = []
        for r in range(0, K):
            temp.append(momentum_returns[k-r])
        averaged_momentum_returns.append(np.sum(temp)/K)
    return averaged_momentum_returns